In [ ]:
import nni
import torch
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import nni.retiarii.nn.pytorch as nn
import nni.retiarii.strategy as strategy

from collections import OrderedDict

from nni.experiment import Experiment
from nni.retiarii import model_wrapper
from nni.retiarii.evaluator import FunctionalEvaluator
from nni.retiarii.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig

from skimage.metrics import peak_signal_noise_ratio as skimage_psnr

from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from torchvision.datasets import MNIST
from torchvision import datasets, transforms




# original

In [ ]:
@model_wrapper
class SearchSpace(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super().__init__()

        hidden_channels = 64
        ks = nn.ValueChoice([1, 3], label="Kernel Size")
        dl = nn.ValueChoice([1, 3, 5], label="Dilation Rate")
        pd = (ks-1)*dl//2

        activations = OrderedDict([
            ("RelU", nn.ReLU(inplace=True)),
            ("LeakyRelU", nn.LeakyReLU(inplace=True)),
            ("Sigmoid", nn.Sigmoid()),
            ("Selu", nn.SELU(inplace=True)),
            ("PreLU", nn.PReLU()),
            ("SiLU", nn.SiLU(inplace=True)),
        ])

        downsample = OrderedDict([
            ("AvgPool2d", nn.AvgPool2d(kernel_size=2, stride=2)),
            ("MaxPool2d", nn.MaxPool2d(kernel_size=2, stride=2)),
        ])

        upsample = OrderedDict([
            ("Nearest", nn.Upsample(scale_factor=2,mode='nearest')),
            ("Bilinear", nn.Upsample(scale_factor=2,mode='bilinear', align_corners=True)),
            ("Bicubic", nn.Upsample(scale_factor=2,mode='bicubic', align_corners=True))
        ])

        # Conv layer in"
        self.layer1_name = "1 - Conv layer in"
        self.conv1 = nn.Conv2d(in_channels, hidden_channels, kernel_size=ks, padding=pd, dilation=dl)
        self.bn1 = nn.BatchNorm2d(hidden_channels)
        self.act1 = nn.LayerChoice(activations, label=f"{self.layer1_name}: Activation Function 1")
        self.conv2 = nn.Conv2d(hidden_channels, hidden_channels, kernel_size=ks, padding=pd, dilation=dl)
        self.bn2 = nn.BatchNorm2d(hidden_channels)
        self.act2 = nn.LayerChoice(activations, label=f"{self.layer1_name}: Activation Function 2")

        self.first = nn.Sequential(
            self.conv1,self.bn1,self.act1,
            self.conv2, self.bn2,self.act2
        )

        # Encoder 1
        self.layer2_name = "2 - Encoder 1"
        self.downsample1 = nn.LayerChoice(downsample,label="First Pooling Technique")

        self.conv3 = nn.Conv2d(hidden_channels, hidden_channels*2, kernel_size=ks, padding=pd, dilation=dl)
        self.bn3 = nn.BatchNorm2d(hidden_channels*2)
        self.act3 = nn.LayerChoice(activations, label=f"{self.layer2_name}: Activation Function 1")
        self.conv4 = nn.Conv2d(hidden_channels*2, hidden_channels*2, kernel_size=ks, padding=pd, dilation=dl)
        self.bn4 = nn.BatchNorm2d(hidden_channels*2)
        self.act4 = nn.LayerChoice(activations, label=f"{self.layer2_name}: Activation Function 2")

        self.Encoder1 = nn.Sequential(
            self.downsample1,
            self.conv3,self.bn3,self.act3,
            self.conv4,self.bn4,self.act4
        )

        # Decoder 1
        self.layer3_name = "3 - Decoder 1"
        self.upsample1 = nn.LayerChoice(upsample, label="First Upsample Technique")

        self.conv5 = nn.Conv2d(hidden_channels*3, hidden_channels, kernel_size=ks, padding=pd, dilation=dl)
        self.bn5 = nn.BatchNorm2d(hidden_channels)
        self.act5 = nn.LayerChoice(activations, label=f"{self.layer3_name}: Activation Function 1")
        self.conv6 = nn.Conv2d(hidden_channels, hidden_channels, kernel_size=ks, padding=pd, dilation=dl)
        self.bn6 = nn.BatchNorm2d(hidden_channels)
        self.act6 = nn.LayerChoice(activations, label=f"{self.layer3_name}: Activation Function 2")

        self.Decoder1 = nn.Sequential(
            self.conv5, self.bn5, self.act5,
            self.conv6, self.bn6, self.act6
            )

        # Conv layer out
        self.out = nn.Conv2d(hidden_channels, out_channels, kernel_size=ks, padding=pd, dilation=dl)
    
    def crop_tensor(self, target_tensor, tensor):
        target_size = target_tensor.size()[2]  # Assuming height and width are same
        tensor_size = tensor.size()[2]
        delta = tensor_size - target_size
        delta = delta // 2
        return tensor[:, :, delta:tensor_size-delta, delta:tensor_size-delta]
    
    def upsample_and_crop(self, input, skip, upsample, decode):
        upsampled = upsample(input)
        cropped = self.crop_tensor(upsampled, skip)
        return decode(torch.cat([cropped, upsampled], 1))

    def print_info(self, layer_name, tensor):
        separator = "-" * 50
        if layer_name == "Input":
            print(separator)
            print(separator)
        if layer_name == "Output":
            print(f"Layer: {layer_name} -- Conv Layer Out")
            print(f"Output Shape: {tensor.shape}")
            print(separator)
            print(separator)
        else:        
            print(f"Layer: {layer_name}")
            print(f"Output Shape: {tensor.shape}")
            print(separator)
    
    def forward(self, x):
        self.print_info("Input", x)

        x = self.first(x)
        self.print_info("Conv layer in", x)

        x1 = self.Encoder1(x)
        self.print_info("Encoder 1", x1)

        x2 = self.upsample_and_crop(
            x1, # input
            x, # skip
            self.upsample1, # upsample 
            self.Decoder1 # decode
            )
        self.print_info("Decoder 1", x2)

        x3 = self.out(x2)
        self.print_info("Output", x3)

        return x3


In [ ]:
def psnr(image_true, image_test):
    # Convert PyTorch tensors to NumPy arrays
    if torch.is_tensor(image_true):
        image_true = image_true.detach().cpu().numpy()
    if torch.is_tensor(image_test):
        image_test = image_test.detach().cpu().numpy()
    return skimage_psnr(image_true, image_test)

def add_noise(img, noise_factor=0.5):
    """Add random noise to an image."""
    noise = torch.randn_like(img) * noise_factor
    noisy_img = img + noise
    return torch.clamp(noisy_img, 0., 1.)

def deep_image_prior_denoising(model, noisy_img, clean_img, device, optimizer, iterations=3000):
    model.train()
    for iteration in range(iterations):
        optimizer.zero_grad()
        output = model(torch.randn(noisy_img.shape).to(device))
        loss = nn.MSELoss()(output, noisy_img)
        loss.backward()
        optimizer.step()
        if iteration % 250 == 0:
            # Calculate PSNR
            with torch.no_grad():
                denoised_output = model(noisy_img)
                psnr_value = psnr(clean_img, denoised_output)
            print('Iteration: {}\tLoss: {:.6f}\tPSNR: {:.6f} dB'.format(iteration, loss.item(), psnr_value))
            nni.report_intermediate_result(psnr_value)
    return output

def evaluate_denoising(denoised_img, clean_img):
    # We no longer need the model in an eval state or any forward pass here
    # because the denoised image is already generated and passed to the function.
    return psnr(clean_img, denoised_img)

def main_evaluation(model_cls):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    # Instantiate model
    model = model_cls().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    transform = transforms.Compose([transforms.ToTensor()])
    dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    img, _ = dataset[0]  # Original clean image
    noisy_img = add_noise(img).unsqueeze(0).to(device)  # Noisy version of image

    # Denoise the image for a set number of iterations
    denoised_img = deep_image_prior_denoising(model, noisy_img, img.unsqueeze(0).to(device), device, optimizer)

    # Evaluate the PSNR of the denoised image
    psnr_value = evaluate_denoising(denoised_img, img.unsqueeze(0).to(device))
    print('PSNR: {:.6f} dB'.format(psnr_value))

    # Report final PSNR to NNI
    nni.report_final_result(psnr_value.item())


# blocks

In [ ]:
class Convolutions(nn.Module):
    def __init__(self, in_channels, out_channels, ks, pd, dl, activations, layer_name):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=ks, padding=pd, dilation=dl)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.act1 = nn.LayerChoice(activations, label=f'{layer_name}: activation 1')
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=ks, padding=pd, dilation=dl)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.act2 = nn.LayerChoice(activations, label=f'{layer_name}: activation 2')

        self.seq = nn.Sequential(self.conv1, self.bn1, self.act1, self.conv2, self.bn2, self.act2)

    def forward(self, x):
        return self.seq(x)

@model_wrapper
class SearchSpace(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super().__init__()

        hidden_channels = 64
        ks = nn.ValueChoice([1, 3], label="Kernel Size")
        dl = nn.ValueChoice([1, 3, 5], label="Dilation Rate")
        pd = (ks-1)*dl//2
        pdd = ks // 2

        activations = OrderedDict([
            ("RelU", nn.ReLU(inplace=True)),
            ("LeakyRelU", nn.LeakyReLU(inplace=True)),
            ("Sigmoid", nn.Sigmoid()),
            ("Selu", nn.SELU(inplace=True)),
            ("PreLU", nn.PReLU()),
            ("SiLU", nn.SiLU(inplace=True)),
        ])

        downsample = OrderedDict([
            ("AvgPool2d", nn.AvgPool2d(kernel_size=2, stride=2)),
            ("MaxPool2d", nn.MaxPool2d(kernel_size=2, stride=2)),
        ])

        upsample = OrderedDict([
            ("Nearest", nn.Upsample(scale_factor=2,mode='nearest')),
            ("Bilinear", nn.Upsample(scale_factor=2,mode='bilinear', align_corners=True)),
            ("Bicubic", nn.Upsample(scale_factor=2,mode='bicubic', align_corners=True))
        ])

        encoderconvs1 = OrderedDict([
            ("Conv2d", nn.Conv2d(hidden_channels, hidden_channels*2, kernel_size=ks, padding=pd, dilation=dl)),
            ("DepthwiseSeparable", nn.Sequential(
                nn.Conv2d(hidden_channels, hidden_channels, kernel_size=ks, padding=pdd, dilation=dl, groups=hidden_channels),
                nn.Conv2d(hidden_channels, hidden_channels*2, kernel_size=1, padding=0, dilation=1)
                )
            )
        ])

        # encoderconvs2 = OrderedDict([
        #     ("Conv2d", nn.Conv2d(hidden_channels*2, hidden_channels*2, kernel_size=ks, padding=pd, dilation=dl)),
        #     ("DepthwiseSeparable", nn.Sequential(
        #         nn.Conv2d(hidden_channels*2, hidden_channels*2, kernel_size=ks, padding=pdd, dilation=dl, groups=hidden_channels),
        #         nn.Conv2d(hidden_channels*2, hidden_channels*2, kernel_size=1, padding=0, dilation=1)
        #         )
        #     ),
        #     ("Depthwise", nn.Conv2d(hidden_channels*2, hidden_channels*2, kernel_size=1, padding=0, dilation=1, groups=hidden_channels))
        # ])

        # decoderconvs1 = OrderedDict([
        #     ("Conv2d", nn.Conv2d(hidden_channels*3, hidden_channels, kernel_size=ks, padding=pd, dilation=dl)),
        #     ("DepthwiseSeparable", nn.Sequential(
        #         nn.Conv2d(hidden_channels*3, hidden_channels*3, kernel_size=ks, padding=pdd, dilation=dl, groups=hidden_channels*3),
        #         nn.Conv2d(hidden_channels, hidden_channels, kernel_size=1, padding=0, dilation=1)
        #         )
        #     )
        # ])

        # decoderconvs2 = OrderedDict([
        #     ("Conv2d", nn.Conv2d(hidden_channels, hidden_channels, kernel_size=ks, padding=pd, dilation=dl)),
        #     ("DepthwiseSeparable", nn.Sequential(
        #         nn.Conv2d(hidden_channels, hidden_channels, kernel_size=ks, padding=pdd, dilation=dl, groups=hidden_channels),
        #         nn.Conv2d(hidden_channels, hidden_channels, kernel_size=1, padding=0, dilation=1)
        #         )
        #     ),
        #     ("Depthwise", nn.Conv2d(hidden_channels, hidden_channels, kernel_size=1, padding=0, dilation=1, groups=hidden_channels))
        # ])

        # Conv layer in"
        self.first = Convolutions(in_channels, hidden_channels, ks, pd, dl, activations, "1 - Conv layer in")

        # Encoder 1
        self.layer2_name = "2 - Encoder 1"
        self.downsample1 = nn.LayerChoice(downsample,label=f'{self.layer2_name}: Downsampling Technique')
        self.Encoder1 = nn.Sequential(
            self.downsample1,
            Convolutions(hidden_channels, hidden_channels*2, ks, pd, dl, activations, self.layer2_name),
        )

        # Decoder 1
        self.layer3_name = "3 - Decoder 1"
        self.upsample1 = nn.LayerChoice(upsample, label=f"{self.layer3_name}: Upsampling Technique")
        self.Decoder1 = Convolutions(hidden_channels*3, hidden_channels, ks, pd, dl, activations, self.layer3_name)

        # Conv layer out
        self.out = nn.Conv2d(hidden_channels, out_channels, kernel_size=ks, padding=pd, dilation=dl)
    
    def crop_tensor(self, target_tensor, tensor):
        target_size = target_tensor.size()[2]  # Assuming height and width are same
        tensor_size = tensor.size()[2]
        delta = tensor_size - target_size
        delta = delta // 2
        return tensor[:, :, delta:tensor_size-delta, delta:tensor_size-delta]
    
    def upsample_and_crop(self, input, skip, upsample, decode):
        upsampled = upsample(input)
        cropped = self.crop_tensor(upsampled, skip)
        return decode(torch.cat([cropped, upsampled], 1))

    def print_info(self, layer_name, tensor):
        separator = "-" * 50
        if layer_name == "Input":
            print(separator)
            print(separator)
        if layer_name == "Output":
            print(f"Layer: {layer_name} -- Conv Layer Out")
            print(f"Output Shape: {tensor.shape}")
            print(separator)
            print(separator)
        else:        
            print(f"Layer: {layer_name}")
            print(f"Output Shape: {tensor.shape}")
            print(separator)
    
    def forward(self, x):
        self.print_info("Input", x)

        x = self.first(x)
        self.print_info("Conv layer in", x)

        x1 = self.Encoder1(x)
        self.print_info("Encoder 1", x1)

        x2 = self.upsample_and_crop(
            x1, # input
            x, # skip
            self.upsample1, # upsample 
            self.Decoder1 # decode
            )
        self.print_info("Decoder 1", x2)

        x3 = self.out(x2)
        self.print_info("Output", x3)

        return x3


In [ ]:
# search space
model_space = SearchSpace()
evaluator = FunctionalEvaluator(main_evaluation)

# search strategy
search_strategy = strategy.Random(dedup=True)

# experiment
exp = RetiariiExperiment(model_space, evaluator, [], search_strategy)
exp_config = RetiariiExeConfig('local')
exp_config.experiment_name = 'mnist_search'
exp_config.trial_code_directory = 'C:/Users/Public/Public_VS_Code/NAS_test'
exp_config.experiment_working_directory = 'C:/Users/Public/nni-experiments'

exp_config.max_trial_number = 1200   # spawn 4 trials at most
exp_config.trial_concurrency = 2  # will run two trials concurrently

exp_config.trial_gpu_number = 1
exp_config.training_service.use_active_gpu = True

# Execute
exp.run(exp_config, 8081)

In [ ]:
experiment = Experiment.connect(8081)
experiment.stop()

# next test

In [ ]:
class Convolutions(nn.Module):
    def __init__(self, out_channels, activations, convs1, convs2, layer_name):
        super().__init__()

        self.conv1 = nn.LayerChoice(convs1, label=f'{layer_name} - Step 1: convolution 1')
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.act1 = nn.LayerChoice(activations, label=f'{layer_name} - Step 2: activation 1')
        
        self.conv2 = nn.LayerChoice(convs2, label=f'{layer_name} - Step 3: convolution 2')
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.act2 = nn.LayerChoice(activations, label=f'{layer_name} - Step 4: activation 2')

        self.seq = nn.Sequential(self.conv1, self.bn1, self.act1, self.conv2, self.bn2, self.act2)

    def forward(self, x):
        return self.seq(x)

@model_wrapper
class SearchSpace(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super().__init__()

        ks = nn.ValueChoice([1, 5], label="Kernel Size")
        dl = nn.ValueChoice([3], label="Dilation Rate")
        pd = (ks - 1) * dl // 2
        # pdd = ks // 2

        activations = OrderedDict([
            ("RelU", nn.ReLU(inplace=True)),
            # ("LeakyRelU", nn.LeakyReLU(inplace=True)),
            # ("Sigmoid", nn.Sigmoid()),
            # ("Selu", nn.SELU(inplace=True)),
            # ("PreLU", nn.PReLU()),
            # ("SiLU", nn.SiLU(inplace=True)),
        ])

        downsamples = OrderedDict([
            ("AvgPool2d", nn.AvgPool2d(kernel_size=2, stride=2)),
            # ("MaxPool2d", nn.MaxPool2d(kernel_size=2, stride=2)),
        ])

        upsamples = OrderedDict([
            ("Nearest", nn.Upsample(scale_factor=2,mode='nearest')),
            # ("Bilinear", nn.Upsample(scale_factor=2,mode='bilinear', align_corners=True)),
            # ("Bicubic", nn.Upsample(scale_factor=2,mode='bicubic', align_corners=True))
        ])

        # Conv layer in"
        self.layer1_out_channels = 64
        self.convs1 = self.get_conv_ordered_dict(in_channels, self.layer1_out_channels, ks, pd, dl)
        self.convs2 = self.get_conv_ordered_dict(self.layer1_out_channels, self.layer1_out_channels, ks, pd, dl, first=False)
        self.first = Convolutions(self.layer1_out_channels, activations, self.convs1, self.convs2, "1 - Conv layer in")

        # Encoder 1
        self.layer2_name = "2 - Encoder 1"
        self.layer2_in_channels = 64
        self.layer2_out_channels = 128
        self.encoderconvs1 = self.get_conv_ordered_dict(self.layer2_in_channels, self.layer2_out_channels, ks, pd, dl)
        self.encoderconvs2 = self.get_conv_ordered_dict(self.layer2_out_channels, self.layer2_out_channels, ks, pd, dl, first=False)
        self.downsample1 = nn.LayerChoice(downsamples,label=f'{self.layer2_name}: Downsampling Technique')
        self.Encoder1 = nn.Sequential(
            self.downsample1,
            Convolutions(self.layer2_out_channels, activations, self.encoderconvs1, self.encoderconvs2, self.layer2_name),
        )

        # Decoder 1
        self.layer3_name = "3 - Decoder 1"
        self.layer3_in_channels = 64*3
        self.layer3_out_channels = 64
        self.decoderconvs1 = self.get_conv_ordered_dict(self.layer3_in_channels, self.layer3_out_channels, ks, pd, dl)
        self.decoderconvs2 = self.get_conv_ordered_dict(self.layer3_out_channels, self.layer3_out_channels, ks, pd, dl, first=False)
        self.upsample1 = nn.LayerChoice(upsamples, label=f"{self.layer3_name}: Upsampling Technique")
        self.Decoder1 = Convolutions(self.layer3_out_channels, activations, self.decoderconvs1, self.decoderconvs2, self.layer3_name)

        # Conv layer out
        self.out = nn.Conv2d(self.layer3_out_channels, out_channels, kernel_size=ks, padding=pd, dilation=dl)
    
    def forward(self, x):
        self.print_info("Input", x)

        x = self.first(x)
        self.print_info("Conv layer in", x)

        x1 = self.Encoder1(x)
        self.print_info("Encoder 1", x1)

        x2 = self.upsample_and_crop(
            x1, # input
            x, # skip
            self.upsample1, # upsample 
            self.Decoder1 # decode
            )
        self.print_info("Decoder 1", x2)

        x3 = self.out(x2)
        self.print_info("Output", x3)

        return x3
    
    def get_conv_ordered_dict(self, in_channels, out_channels, ks, pd, dl, first=False):
         layers = [
                ("Conv2d", nn.Conv2d(in_channels, out_channels, kernel_size=ks, padding=pd, dilation=dl)),
                ("DepthwiseSeparable", nn.Sequential(
                    nn.Conv2d(in_channels, in_channels, kernel_size=ks, padding=pd, dilation=dl, groups=in_channels),
                    nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0, dilation=1)
                    )
                )
            ]
         if not first:
            layers.append(("Depthwise", nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0, dilation=1)))
         return OrderedDict(layers)

    def crop_tensor(self, target_tensor, tensor):
        target_size = target_tensor.size()[2]  # Assuming height and width are same
        tensor_size = tensor.size()[2]
        delta = tensor_size - target_size
        delta = delta // 2
        return tensor[:, :, delta:tensor_size-delta, delta:tensor_size-delta]
    
    def upsample_and_crop(self, input, skip, upsample, decode):
        upsampled = upsample(input)
        cropped = self.crop_tensor(upsampled, skip)
        return decode(torch.cat([cropped, upsampled], 1))

    def print_info(self, layer_name, tensor):
        separator = "-" * 50
        if layer_name == "Input":
            print(separator)
            print(separator)
        if layer_name == "Output":
            print(f"Layer: {layer_name} -- Conv Layer Out")
            print(f"Output Shape: {tensor.shape}")
            print(separator)
            print(separator)
        else:        
            print(f"Layer: {layer_name}")
            print(f"Output Shape: {tensor.shape}")
            print(separator)


In [ ]:
# search space
model_space = SearchSpace()
evaluator = FunctionalEvaluator(main_evaluation)

# search strategy
search_strategy = strategy.Random(dedup=True)

# experiment
exp = RetiariiExperiment(model_space, evaluator, [], search_strategy)
exp_config = RetiariiExeConfig('local')
exp_config.experiment_name = 'mnist_search'
exp_config.trial_code_directory = 'C:/Users/Public/Public_VS_Code/NAS_test'
exp_config.experiment_working_directory = 'C:/Users/Public/nni-experiments'

exp_config.max_trial_number = 1200   # spawn 4 trials at most
exp_config.trial_concurrency = 2  # will run two trials concurrently

exp_config.trial_gpu_number = 1
exp_config.training_service.use_active_gpu = True

# Execute
exp.run(exp_config, 8081)

In [ ]:
experiment = Experiment.connect(8081)
experiment.stop()